In [2]:
# 🛠️ 1. Installazione delle librerie (solo su Colab)
!pip install transformers accelerate pandas


In [3]:

# ⚙️ 2. Importazione dei moduli
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd
import os

# 🖥️ 3. Configurazione (evita problemi su alcune macchine)
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 🔍 4. Verifica della GPU
print("✅ GPU disponibile:", torch.cuda.is_available())

# 📦 5. Caricamento del modello Phi-3.5-mini-instruct
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    device_map="auto",  # Usa automaticamente la GPU se disponibile
    torch_dtype="auto",
    trust_remote_code=False
)

# 🔤 6. Caricamento del tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    trust_remote_code=True
)

# 🔁 7. Creazione della pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    temperature=0.2,
    top_p=1
)

# 📄 8. Caricamento del file CSV (caricalo manualmente in Colab prima)
from google.colab import files
uploaded = files.upload()  # Carica topic_keywords.csv

df = pd.read_csv("topic_keywords.csv")
topic_groups = df.groupby("Topic")["Word"].apply(list)




✅ GPU disponibile: True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Saving topic_keywords.csv to topic_keywords (1).csv


In [11]:
prompt = (
    "Ti fornisco liste di parole chiave, una per ciascun topic, ottenute dal file 'topic_keywords.csv'.\n"
    "Per ogni lista, suggerisci un'etichetta che rappresenti il tema principale del topic.\n"
    "Evita di usare le stesse parole delle liste. L'etichetta deve essere una singola parola.\n"
    "Scrivi solo una parola per riga, senza testo aggiuntivo, numeri o 'Etichetta:'.\n\n"
)

# Solo i primi 7 topic
for topic_id, words in list(topic_groups.items())[:7]:
    prompt += f"Topic {topic_id} - Parole chiave: {', '.join(words)}\n"

prompt += "\nScrivi una sola parola per ogni topic, una per riga, nello stesso ordine."

# Genera output
output = pipe(prompt)

# Testo generato
text_output = output[0]["generated_text"]

# Stampa su console
print("\nEtichette generate per i topic:")
print(text_output)

# Salvataggio su file di testo
with open("etichette_topic.txt", "w", encoding="utf-8") as f:
    f.write(text_output)



Etichette generate per i topic:
Ti fornisco liste di parole chiave, una per ciascun topic, ottenute dal file 'topic_keywords.csv'.
Per ogni lista, suggerisci un'etichetta che rappresenti il tema principale del topic.
Evita di usare le stesse parole delle liste. L'etichetta deve essere una singola parola.
Scrivi solo una parola per riga, senza testo aggiuntivo, numeri o 'Etichetta:'.

Topic 0 - Parole chiave: room, hotel, stay, staff, not, night, location, good, great, bed
Topic 1 - Parole chiave: resort, beach, food, not, pool, time, good, day, people, go
Topic 2 - Parole chiave: barcelona, hotel, rambla, room, location, metro, stay, good, great, walk
Topic 3 - Parole chiave: florence, hotel, stay, room, breakfast, location, venice, duomo, walk, italy
Topic 4 - Parole chiave: amsterdam, hotel, room, stay, tram, location, canal, breakfast, walk, staff
Topic 5 - Parole chiave: paris, hotel, metro, room, stay, eiffel, location, staff, small, great
Topic 6 - Parole chiave: juan, san, beac